`Author: Victor Radermecker | Date: 11/10/2021 | Project: Master Thesis`

This notebook computes the final demand estimation using the PPR and INDR ratios.

In [1]:
import json
import math
import os

# Importing local libraries
import sys

import branca.colormap as cm
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import *
from tqdm import tqdm

sys.path.insert(0, "../src")
from BrusselsMap import BrusselsMap
from GeoJsonHandler import GeoJsonHandler

pd.set_option("display.max_rows", 500)

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Demand Estimation Computation

#### Proximus data

In [21]:
prox = pd.read_csv("../data/final_demand_simulation/proximusFrequentTrip_RandomlyGenerated.csv")
prox.head()

,Unnamed: 0,originNB,destinationNB,regularTripSample,irregularTripSample
0,1,ALTITUDE 100,ALTITUDE 100,242,4066
1,2,ALTITUDE 100,ANDERLECHT CENTRE - WAYEZ,9666,7490
2,3,ALTITUDE 100,ANNEESSENS,556,8981
3,4,ALTITUDE 100,AUDERGHEM CENTRE,9465,7964
4,5,ALTITUDE 100,AVENUE LEOPOLD III,7479,318


#### Convert to Energy 

In [22]:
dists = pd.read_csv("../data/final_demand_simulation/DemandClusters-ODMatrix-Km.csv").set_index(
    "origin"
)

# Convert number of trips into distance
dists_long = (
    dists.unstack()
    .reset_index()
    .rename(
        columns={"level_0": "destinationNB", "origin": "originNB", 0: "travelDistance"}
    )[["originNB", "destinationNB", "travelDistance"]]
)
prox = pd.merge(prox, dists_long, on=["originNB","destinationNB"], how="outer").fillna(0)

# Convert to Energy using IEA Assumption on EV's consumption (0.22kWh/km)
prox['regularDemand'] = prox['regularTripSample'] * prox['travelDistance']
prox['irregularDemand'] = prox['irregularTripSample'] * prox['travelDistance']
prox['totalDemand'] = prox['regularDemand'] + prox['irregularDemand']

prox['regularEnergyDemand'] = prox['regularDemand'] * 0.22 
prox['irregularEnergyDemand'] = prox['irregularDemand'] * 0.22 

#Aggregate data and save
prox = prox.groupby(by=['destinationNB'])[['regularTripSample','irregularTripSample', 'travelDistance', 'regularDemand', 'irregularDemand','totalDemand', 'regularEnergyDemand','irregularEnergyDemand']].sum().reset_index()
prox.rename(columns={'destinationNB':'NAME_FRE'}, inplace=True)
prox.to_csv('../data/final_demand_simulation/DemandEnergy.csv', index=False)

#### PPR Ratio

In [23]:
# Loading parking and population data
ppr = pd.read_csv("../data/final_data/parkings_ratio.csv")
ppr.head()

,NAME_FRE,AREA,DENS,POP,AVG_HOUS,HOUSEHOLDS,NOT REGLEMENTED,RESERVED,REGLEMENTED,PRIVATE,PRIVATE/HOUS
0,CONSCIENCE,0.469490,17518.92,8225,2.33,3530,0,57,1260,551,0.156091
1,HELMET,0.718634,17717.47,12732,2.54,5013,0,81,2315,552,0.110114
2,VIEUX LAEKEN OUEST,0.499646,16781.39,8385,2.55,3288,0,90,996,310,0.094282
3,VIEUX LAEKEN EST,1.040450,18272.53,19012,2.61,7284,0,211,2858,664,0.091159
4,INDUSTRIE NORD,6.733601,261.99,1764,2.18,809,1277,26,248,218,0.269468


#### INDR Ratio

In [27]:
# Loading parking and population data
indr = pd.read_csv("../data/final_data/INDR_distances.csv")
indr['ratio'] = indr['ratio'] / 100
indr.head()

,NAME_FRE,dist,ratio
0,CONSCIENCE,803.190416,0.198113
1,HELMET,852.081566,0.205969
2,VIEUX LAEKEN OUEST,665.214251,0.175941
3,VIEUX LAEKEN EST,1030.570800,0.234652
4,INDUSTRIE NORD,3110.748195,0.568927


### Merge datasets together

In [28]:
prox = pd.merge(prox, ppr[['NAME_FRE','PRIVATE/HOUS']], on="NAME_FRE", how="outer")
prox = pd.merge(prox, indr[['NAME_FRE','ratio']], on="NAME_FRE", how="outer")
prox.rename(columns={'ratio':'INDR_Ratio', 'PRIVATE/HOUS':'PPR_Ratio'}, inplace=True)
prox = prox.fillna(0)
prox.head()

,NAME_FRE,regularTripSample,irregularTripSample,travelDistance,regularDemand,irregularDemand,totalDemand,regularEnergyDemand,irregularEnergyDemand,PPR_Ratio,INDR_Ratio,PPR_Ratio,INDR_Ratio
0,ALTITUDE 100,741855.0,674969.0,1040.854,5157861.733,4886114.733,1.004398e+07,1.134730e+06,1.074945e+06,0.145925,19.277802,0.145925,0.192778
1,ANDERLECHT CENTRE - WAYEZ,696621.0,642094.0,1172.197,6022659.886,5447877.736,1.147054e+07,1.324985e+06,1.198533e+06,0.127411,26.245925,0.127411,0.262459
2,ANNEESSENS,696328.0,663316.0,852.715,3922112.759,3707412.485,7.629525e+06,8.628648e+05,8.156307e+05,0.066221,17.467893,0.066221,0.174679
3,AUDERGHEM CENTRE,718066.0,719010.0,1560.934,7689852.337,7445426.328,1.513528e+07,1.691768e+06,1.637994e+06,0.212121,24.529868,0.212121,0.245299
4,AVENUE LEOPOLD III,696597.0,638930.0,1425.105,6364367.186,5597311.181,1.196168e+07,1.400161e+06,1.231408e+06,0.148261,24.835558,0.148261,0.248356


In [ ]:
#Load geodata
r_path = r"../data/final_data/sector.json"
path = os.path.join(os.getcwd(), r_path)
sectors = GeoJsonHandler(path, "NAME_FRE")